In [0]:
import numpy as np
import time

import tensorflow as tf

kernel_a = np.random.randint(low = -10, high = 10, size=(3,3));
input_a = np.random.randint(low = -10, high = 10, size=(32,32));


In [2]:
#Normal convolution

k = tf.convert_to_tensor(kernel_a, np.float32);
i = tf.convert_to_tensor(input_a, np.float32);

kernelt = tf.reshape(k, [3, 3, 1, 1], name='kernel')
imaget  = tf.reshape(i, [1, 32, 32, 1], name='image')

ans_conv_tens = tf.squeeze(tf.nn.conv2d(imaget, kernelt, [1, 1, 1, 1], "VALID"))
start_time = time.time()

# VALID means no padding
with tf.Session() as sess:
   ans_conv = sess.run(ans_conv_tens)

print(ans_conv)
exec_time_conv = time.time() - start_time

[[-142.  -52.   -5.  178. -225.   10.  -58.  181.  110. -143.  -34.  -15.
   148.  -90.   32.   28.  -36.  126. -191.  -47.  241. -156.    0.   57.
   104.   16. -106.  -89. -108.  -89.]
 [-248.   80.   94.   99.  -62.    0.  233. -131. -133.  -68.  118.  153.
   169.  -67. -129.   52.  -52.  152.   55.  -38.  149. -154.  129.   64.
  -167. -120.   58.  -15.  -29.  219.]
 [  -4.   40. -126.  111.   45.  -78.  -91.   97. -139.   33.  214. -113.
  -104.   22.   -9.   15. -169.  128.    8.  -92.  -37.  -41.  124. -144.
   179.  129.    4. -185.   10.  157.]
 [  12. -296.  141.   15.  -72.  -98. -106.  185.  156.  -28. -163.  -59.
  -104.   85.  141.  -19.   69. -121.  149. -123.  -81. -176.  224. -168.
   157.   21.  -30.   10. -137.   24.]
 [  21.  -14. -108.  311. -220.  -57.   46.   60.  119.  -39. -120.  282.
     1. -170.   56.  -64.    2.   24.   84. -126.  141.  -74.  188. -160.
   -38.  -81.   72.   90. -264.  271.]
 [ 206.   91. -274.  146.  -14.   -7.  -73.   93.  -76.   78.  12

In [0]:
from numpy.lib.stride_tricks import as_strided as ast

def norm_shape(shape):
    '''
    Normalize numpy array shapes so they're always expressed as a tuple,
    even for one-dimensional shapes.
     
    Parameters
        shape - an int, or a tuple of ints
     
    Returns
        a shape tuple
    '''
    try:
        i = int(shape)
        return (i,)
    except TypeError:
        # shape was not a number
        pass
 
    try:
        t = tuple(shape)
        return t
    except TypeError:
        # shape was not iterable
        pass
     
    raise TypeError('shape must be an int, or a tuple of ints')
  
def _create_tiles(a,ws,ss = None,flatten = False):
    '''
    Return a sliding window over a in any number of dimensions
     
    Parameters:
        a  - an n-dimensional numpy array
        ws - an int (a is 1D) or tuple (a is 2D or greater) representing the size
             of each dimension of the window
        ss - an int (a is 1D) or tuple (a is 2D or greater) representing the
             amount to slide the window in each dimension. If not specified, it
             defaults to ws.
        flatten - if True, all slices are flattened, otherwise, there is an
                  extra dimension for each dimension of the input.
     
    Returns
        an array containing each n-dimensional window from a
    '''
     
    if None is ss:
        # ss was not provided. the windows will not overlap in any direction.
        ss = ws
    ws = norm_shape(ws)
    ss = norm_shape(ss)
     
    # convert ws, ss, and a.shape to numpy arrays so that we can do math in every
    # dimension at once.
    ws = np.array(ws)
    ss = np.array(ss)
    shape = np.array(a.shape)
     
     
    # ensure that ws, ss, and a.shape all have the same number of dimensions
    ls = [len(shape),len(ws),len(ss)]
    if 1 != len(set(ls)):
        raise ValueError(\
        'a.shape, ws and ss must all have the same length. They were %s' % str(ls))
     
    # ensure that ws is smaller than a in every dimension
    if np.any(ws > shape):
        raise ValueError(\
        'ws cannot be larger than a in any dimension.\
 a.shape was %s and ws was %s' % (str(a.shape),str(ws)))
     
    # how many slices will there be in each dimension?
    newshape = norm_shape(((shape - ws) // ss) + 1)
    # the shape of the strided array will be the number of slices in each dimension
    # plus the shape of the window (tuple addition)
    newshape += norm_shape(ws)
    # the strides tuple will be the array's strides multiplied by step size, plus
    # the array's strides (tuple addition)
    newstrides = norm_shape(np.array(a.strides) * ss) + a.strides
    strided = ast(a,shape = newshape,strides = newstrides)
    if not flatten:
        return strided
     
    # Collapse strided so that it has one more dimension than the window.  I.e.,
    # the new array is a flat list of slices.
    meat = len(ws) if ws.shape else 0
    firstdim = (np.product(newshape[:-meat]),) if ws.shape else ()
    dim = firstdim + (newshape[-meat:])
    # remove any dimensions with size 1
    dim = filter(lambda i : i != 1,dim)
    return strided.reshape(dim)

In [0]:
#Winograd - prereq
# Using F(2x2, 3x3)

import numpy as np
import time

# Block size is m+r-1 = 2+3-1 = 4, stride length = r-1 = 2;
tilesA = _create_tiles(a = input_a, ws = (4, 4), ss = (2,2)) #returns (15x15 array of tiles)
tilesB = tilesA.reshape(1,15*15,4,4); #flatten array
tiles = tilesB[0];

# defined in the paper
G = np.array([[1, 0, 0], [0.5, 0.5, 0.5], [0.5, -0.5, 0.5], [0, 0, 1]])
Bt = np.array([[1, 0, -1, 0],[0, 1, 1, 0], [0, -1, 1, 0], [0, 1, 0, -1]])
At = np.array(([1, 1, 1, 0], [0, 1, -1, -1]))

In [5]:
#Winograd
start_time = time.time()

lhs = np.dot(np.dot(G, kernel_a), G.T)

ans_wino = np.zeros(shape = (30,30))

for i1 in range(0,15):
  for i2 in range(0,15):
    ti = tiles[i1*15 + i2];
    rhsi = np.dot(np.dot(Bt,ti), Bt.T)
    insidei = lhs * rhsi
    ansi = np.dot(np.dot(At, insidei), At.T)
    #print(ansi)
    ans_wino[2*i1:2*i1+2,2*i2:2*i2+2] += ansi

print(ans_wino)
exec_time_wino = time.time() - start_time

[[-142.  -52.   -5.  178. -225.   10.  -58.  181.  110. -143.  -34.  -15.
   148.  -90.   32.   28.  -36.  126. -191.  -47.  241. -156.    0.   57.
   104.   16. -106.  -89. -108.  -89.]
 [-248.   80.   94.   99.  -62.    0.  233. -131. -133.  -68.  118.  153.
   169.  -67. -129.   52.  -52.  152.   55.  -38.  149. -154.  129.   64.
  -167. -120.   58.  -15.  -29.  219.]
 [  -4.   40. -126.  111.   45.  -78.  -91.   97. -139.   33.  214. -113.
  -104.   22.   -9.   15. -169.  128.    8.  -92.  -37.  -41.  124. -144.
   179.  129.    4. -185.   10.  157.]
 [  12. -296.  141.   15.  -72.  -98. -106.  185.  156.  -28. -163.  -59.
  -104.   85.  141.  -19.   69. -121.  149. -123.  -81. -176.  224. -168.
   157.   21.  -30.   10. -137.   24.]
 [  21.  -14. -108.  311. -220.  -57.   46.   60.  119.  -39. -120.  282.
     1. -170.   56.  -64.    2.   24.   84. -126.  141.  -74.  188. -160.
   -38.  -81.   72.   90. -264.  271.]
 [ 206.   91. -274.  146.  -14.   -7.  -73.   93.  -76.   78.  12

In [6]:
print("--- Does winograd answer match with normal convolution ? %s ---" % np.array_equiv(ans_conv, ans_wino))
print("--- Time with normal convolution %s ---" % exec_time_conv)
print("--- Time with winograd convolution %s ---" % exec_time_wino)


--- Does winograd answer match with normal convolution ? True ---
--- Time with normal convolution 0.013360261917114258 ---
--- Time with winograd convolution 0.022821664810180664 ---
